### Knock61: Directory 生成をして機械学習用 Data を読み込もう

In [1]:
# Directory 作成
import os
data_dir = 'data'
input_dir = os.path.join(data_dir, '0_input')
output_dir = os.path.join(data_dir, '1_output')
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [2]:
# 機械学習用 Data の読み込み
import pandas as pd
ml_data_file = 'ml_base_data.csv'
ml_data = pd.read_csv(os.path.join(input_dir, ml_data_file))
ml_data.head(3)

store_name  y_weekday  y_weekend  order  order_fin  order_cancel  \
0      あきる野店        1.0        0.0   1147        945           202   
1     さいたま南店        1.0        1.0   1504       1217           287   
2     さいたま緑店        1.0        1.0   1028        847           181   

   order_delivery  order_takeout  order_weekday  order_weekend  ...  \
0             841            306            844            303  ...   
1            1105            399           1104            400  ...   
2             756            272            756            272  ...   

   order_time_14  order_time_15  order_time_16  order_time_17  order_time_18  \
0            101             95            107            106            100   
1            143            142            137            130            113   
2             95            102             82             90             93   

   order_time_19  order_time_20  order_time_21  delta_avg  year_month  
0            108            109             96  34.110053      201904  
1            140            132            155  35.337716      201904  
2             95             95             84  34.291617      201904  

[3 rows x 23 columns]

### Knock62: Categorical 変数の対応をしよう
- One-hot-encoding: Categorical 変数を、特定の Category に属していたら 1 の Flag を立てる形式

In [3]:
from IPython.display import display

# One-hot-encoding
category_data = pd.get_dummies(ml_data['store_name'], prefix='store', prefix_sep='_')
display(category_data.head(3))

store_あきる野店  store_さいたま南店  store_さいたま緑店  store_さいたま西店  store_つくば店  \
0            1             0             0             0           0   
1            0             1             0             0           0   
2            0             0             1             0           0   

   store_三浦店  store_三鷹店  store_上尾店  store_上野店  store_世田谷店  ...  store_駒込店  \
0          0          0          0          0           0  ...          0   
1          0          0          0          0           0  ...          0   
2          0          0          0          0           0  ...          0   

   store_高円寺店  store_高島平店  store_高崎店  store_高座店  store_高津店  store_高田馬場店  \
0           0           0          0          0          0            0   
1           0           0          0          0          0            0   
2           0           0          0          0          0            0   

   store_鴻巣店  store_鶴見店  store_麻生店  
0          0          0          0  
1          0          0          0  
2          0          0          0  

[3 rows x 195 columns]

- pandas の `get_dummies()` を使用すると簡単に One-hot-encoding ができる。
- **多重共線性の防止**: Categorical 変数は１列消すことが一般的。（すべての Flag が 0 だった場合に削除した変数と情報が特定できる為）
- 元の Data に結合する際に、One-hot-encoding の元となった変数は削除する。

In [4]:
# Categorical 変数の結合
del category_data['store_麻生店']
del ml_data['year_month']
del ml_data['store_name']
ml_data = pd.concat([ml_data, category_data], axis=1)
ml_data.columns

Index(['y_weekday', 'y_weekend', 'order', 'order_fin', 'order_cancel',
       'order_delivery', 'order_takeout', 'order_weekday', 'order_weekend',
       'order_time_11',
       ...
       'store_駒沢店', 'store_駒込店', 'store_高円寺店', 'store_高島平店', 'store_高崎店',
       'store_高座店', 'store_高津店', 'store_高田馬場店', 'store_鴻巣店', 'store_鶴見店'],
      dtype='object', length=215)

One-hot-encoding を行なうと、情報としてはわかりにくく、他で使用する際に使いにくくなる為、機械学習に投入する直前で対応すると良い。

### Knock63: 学習 Data と Test data を分割しよう
- 機械学習の目的は、未知な Data に対応せること。
- 未知な Data に対応できる Model が汎用的な良い Model とされ、**汎用性が高い** と表現される。
- 全 Data を Model 構築に使用してしまうと、その Model は未知な Data に対応できるか評価できなくなってしまうため、
    - 学習 Data
    - Test data
    に **分割** する。

#### 学習 Data と Test data の分割
学習 Data と Test data の分割比率に正解はなく、試行錯誤する要素の１つだが、
- 7:3
- 75:25
- 8:2

あたりがよく使用される。

Sample 数が少ないと分割比率による精度の差が顕著にでてくる。その場合、**交差検証** などの分割手法で正しく精度を検証することが重要になってくる。

In [5]:
# 学習 Data と Test data の分割
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(ml_data, test_size=0.3, random_state=0)
print(f"Train: {len(train_data)}件 / Test: {len(test_data)}")
print(f"Weekday Train0: {len(train_data.loc[train_data['y_weekday'] == 0])}件")
print(f"Weekday Train1: {len(train_data.loc[train_data['y_weekday'] == 1])}件")
print(f"Weekday Test0: {len(test_data.loc[test_data['y_weekday'] == 0])}件")
print(f"Weekday Test1: {len(test_data.loc[test_data['y_weekday'] == 1])}件")

print(f"Weekend Train0: {len(train_data.loc[train_data['y_weekend'] == 0])}件")
print(f"Weekend Train1: {len(train_data.loc[train_data['y_weekend'] == 1])}件")
print(f"Weekend Test0: {len(test_data.loc[test_data['y_weekend'] == 0])}件")
print(f"Weekend Test1: {len(test_data.loc[test_data['y_weekend'] == 1])}件")

Train: 1501件 / Test: 644
Weekday Train0: 685件
Weekday Train1: 816件
Weekday Test0: 290件
Weekday Test1: 354件
Weekend Train0: 708件
Weekend Train1: 793件
Weekend Test0: 295件
Weekend Test1: 349件


### Knock64: １つの Model を構築しよう

In [6]:
# 説明変数, 目的変数の作成
X_cols = list(train_data.columns)
X_cols.remove('y_weekday')
X_cols.remove('y_weekend')
target_y = 'y_weekday'
y_train = train_data[target_y]
X_train = train_data[X_cols]
y_test = test_data[target_y]
X_test = test_data[X_cols]
display(y_train.head(3))
display(X_train.head(3))

1137    1.0
971     0.0
1983    1.0
Name: y_weekday, dtype: float64

order  order_fin  order_cancel  order_delivery  order_takeout  \
1137    977        809           168             724            253   
971    1099        904           195             816            283   
1983    966        794           172             724            242   

      order_weekday  order_weekend  order_time_11  order_time_12  \
1137            685            292            102             88   
971             779            320             99            102   
1983            671            295             80             95   

      order_time_13  ...  store_駒沢店  store_駒込店  store_高円寺店  store_高島平店  \
1137             84  ...          0          0           0           0   
971             101  ...          0          0           0           0   
1983             87  ...          0          0           0           0   

      store_高崎店  store_高座店  store_高津店  store_高田馬場店  store_鴻巣店  store_鶴見店  
1137          0          0          0            0          0          0  
971           0          0          0            1          0          0  
1983          0          0          0            0          0          0  

[3 rows x 213 columns]

In [7]:
# 決定木 Model の構築
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

- Model を定義して、`fit()` するだけで Model が構築される。
- Model を定義する際に、**乱数種** の固定を忘れない。

### Knock65: 評価を実施してみよう
Model 構築ができたら、次は評価

In [8]:
# 構築した Model での予測結果
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
display(y_pred_test)

array([0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
       1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
       0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1.,
       0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
       1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1.,
       0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0.,
       0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0.,
       1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 0.

- `model.predict()` だけで構築した Model での予測結果を取得できる。
- 1 は Weekday の Order 数が増加すると予測している。

#### 数字での評価
評価の数字としての Orthodox は
- **正解率**: (TN + TP) / (TN + FP + FN + TP)
- **F値**: 再現率と適合率の調和平均
- **再現率**: TP / (FN + TP)
- **適合率**: TP / (FP + TP)

これらは**混合行列**を書くと理解しやすい。

In [9]:
# 時間帯別 order 数の集計
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)
f1_train = f1_score(y_train, y_pred_train)
f1_test = f1_score(y_test, y_pred_test)
recall_train = recall_score(y_train, y_pred_train)
recall_test = recall_score(y_test, y_pred_test)
precision_train = precision_score(y_train, y_pred_train)
precision_test = precision_score(y_test, y_pred_test)
print(f"【正解率】Train: {round(acc_train, 2)} Test: {round(acc_test, 2)}")
print(f"【F値】Trains: {round(f1_train, 2)} Test: {round(f1_test, 2)}")
print(f"【制限率】Train: {round(recall_train, 2)} Test: {round(recall_test, 2)}")
print(f"【適合率】Train: {round(precision_train, 2)} Test: {round(precision_test, 2)}")

【正解率】Train: 1.0 Test: 0.82
【F値】Trains: 1.0 Test: 0.84
【制限率】Train: 1.0 Test: 0.82
【適合率】Train: 1.0 Test: 0.86


`scikit-learn.metrics` を使用すると簡単に算出できる。

学習が 1.0 は、学習 Data に適合しすぎている**過学習**な状態。

#### 過学習
- 未知の Data に対応できない Model
- 過学習 Model より、学習 Data の精度が低くても、学習 Data と Test data の精度の差が小さい方が良い Model になる。

In [10]:
# 混同行列の表示
print(confusion_matrix(y_train, y_pred_train))
print(confusion_matrix(y_test, y_pred_test))

[[685   0]
 [  0 816]]
[[241  49]
 [ 64 290]]


`scikit-learn.metrics` を使用して簡単に出力できる。
- FP と FN が 0 となり、全部正解している場合、**過学習**と判断できる。

In [11]:
# 混同行列 Data の格納
tn_train, fp_train, fn_train, tp_train = confusion_matrix(y_train, y_pred_train).ravel()
tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
print(f"【混同行列】Train: {tn_train}, {fp_train}, {fn_train}, {tp_train}")
print(f"【混同行列】Test: {tn_test}, {fp_test}, {fn_test}, {tp_test}")

【混同行列】Train: 685, 0, 0, 816
【混同行列】Test: 241, 49, 64, 290


In [12]:
# 精度指標の Data 化
score_train = pd.DataFrame({'DataCategory': ['train'], 'acc': [acc_train], 'f1': [f1_train], 'recall': [recall_train],
                            'precision': [precision_train], 'tp': [tp_train], 'fn': [fn_train], 'fp': [fp_train],
                            'tn': [tn_train]})
score_test = pd.DataFrame({'DataCategory': ['test'], 'acc': [acc_train], 'f1': [f1_test], 'recall': [recall_test],
                           'precision': [precision_test], 'tp': [tp_test], 'fn': [fn_test], 'fp': [fp_test],
                           'tn': [tn_test]})
score = pd.concat([score_train, score_test], ignore_index=True)
score

DataCategory  acc        f1    recall  precision   tp  fn  fp   tn
0        train  1.0  1.000000  1.000000   1.000000  816   0   0  685
1         test  1.0  0.836941  0.819209   0.855457  290  64  49  241

### Knock66: Model の重要度を確認してみよう

決定木等の**木系 Algorithm** は、`feature_importances` を使うと、構築した Model に寄与している変数が取得できる。

In [13]:
# Model の重要度
importance = pd.DataFrame({'cols': X_train.columns, 'importance': model.feature_importances_})
importance = importance.sort_values('importance', ascending=False)
importance.head(10)

cols  importance
5    order_weekday    0.369241
6    order_weekend    0.346013
18       delta_avg    0.027430
2     order_cancel    0.026031
12   order_time_16    0.024161
8    order_time_12    0.023256
3   order_delivery    0.020037
17   order_time_21    0.018936
10   order_time_14    0.017932
11   order_time_15    0.017163

### Knock67: Model 構築から評価までを関数化しよう

In [14]:
def make_model_and_eval(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    acc_train = accuracy_score(y_train, y_pred_train)
    acc_test = accuracy_score(y_test, y_pred_test)
    f1_train = f1_score(y_train, y_pred_train)
    f1_test = f1_score(y_test, y_pred_test)
    recall_train = recall_score(y_train, y_pred_train)
    recall_test = recall_score(y_test, y_pred_test)
    precision_train = precision_score(y_train, y_pred_train)
    precision_test = precision_score(y_test, y_pred_test)
    tn_train, fp_train, fn_train, tp_train = confusion_matrix(y_train, y_pred_train).ravel()
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    score_train = pd.DataFrame(
        {'DataCategory': ['train'], 'acc': [acc_train], 'f1': [f1_train], 'recall': [recall_train],
         'precision': [precision_train], 'tp': [tp_train], 'fn': [fn_train], 'fp': [fp_train], 'tn': [tn_train]})
    score_test = pd.DataFrame(
        {'DataCategory': ['test'], 'acc': [acc_test], 'f1': [f1_test], 'recall': [recall_test],
         'precision': [precision_test], 'tp': [tp_test], 'fn': [fn_test], 'fp': [fp_test], 'tn': [tn_test]})
    score = pd.concat([score_train, score_test], ignore_index=True)
    importance = pd.DataFrame({'cols': X_train.columns, 'importance': model.feature_importances_})
    importance = importance.sort_values('importance', ascending=False)
    cols = pd.DataFrame({'X_cols': X_train.columns})
    display(score)
    return score, importance, model, cols

In [15]:
# 関数を使用した決定木 Model の構築および評価
model = DecisionTreeClassifier(random_state=0)
score, importance, model, cols = make_model_and_eval(model, X_train, X_test, y_train, y_test)

DataCategory       acc        f1    recall  precision   tp  fn  fp   tn
0        train  1.000000  1.000000  1.000000   1.000000  816   0   0  685
1         test  0.824534  0.836941  0.819209   0.855457  290  64  49  241

### Knock68: Model file や評価結果を出力しよう
- Model 構築は、試行錯誤が多いため、何度も評価結果や Model file を出力することが多い。
- 上記の理由から上書きしないような工夫が必要。

In [16]:
# 出力 Directory の作成
import datetime

now = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
target_output_dir_name = 'results_' + now
target_output_dir = os.path.join(output_dir, target_output_dir_name)
os.makedirs(target_output_dir, exist_ok=True)
print(target_output_dir)

data\1_output\results_20220831063031


In [17]:
# 評価結果と Model file の出力
score_name = 'score.csv'
importance_name = 'importance.csv'
cols_name = 'X_cols.csv'
model_name = 'model.pickle'
score_path = os.path.join(target_output_dir, score_name)
importance_path = os.path.join(target_output_dir, importance_name)
cols_path = os.path.join(target_output_dir, cols_name)
model_path = os.path.join(target_output_dir, model_name)

score.to_csv(score_path, index=False)
importance.to_csv(importance_path, index=False)
cols.to_csv(cols_path, index=False)

import pickle

with open(model_path, mode='wb') as f:
    pickle.dump(model, f, protocol=2)